In [44]:
# context vars created in a coroutine are available to others
import contextvars

async def print_context(k, v):
    contextvars.ContextVar(k).set(v)
    print(list(contextvars.copy_context().items()))
    
async def main():
    await print_context('k1', 'v1')
    await print_context('k2', 'v2')
    
await main()

[(<ContextVar name='k1' at 0x109a47530>, 'v1')]
[(<ContextVar name='k2' at 0x109a47650>, 'v2'), (<ContextVar name='k1' at 0x109a47530>, 'v1')]


In [8]:
# changes to a contextvar persist after exiting coroutine
import contextvars
import copy
import asyncio

c = contextvars.ContextVar('c')
c.set({})


def contextupdate(k, v):
    d = copy.copy(c.get())
    d[k] = v
    c.set(d)
    
async def print_context(k, v):
    contextupdate(k, v)
    print(list(contextvars.copy_context().items()))
    
async def main():
    r1 = print_context('k1', 'v1')
    r2 = print_context('k2', 'v2')
    
    # if we block on r1, `set`s persist in context to second task
    # await r1; await r2
    ## [(<ContextVar name='c' at 0x1096fe590>, {'k1': 'v1'})]
    ## [(<ContextVar name='c' at 0x1096fe590>, {'k1': 'v1', 'k2': 'v2'})]

    # ...but if not, second task gets context at the time of the call
    await asyncio.gather(r1, r2)
    #[(<ContextVar name='c' at 0x1096febf0>, {'k1': 'v1'})]
    # [(<ContextVar name='c' at 0x1096febf0>, {'k2': 'v2'})]
    
await main()

[(<ContextVar name='c' at 0x1096febf0>, {'k1': 'v1'})]
[(<ContextVar name='c' at 0x1096febf0>, {'k2': 'v2'})]


In [37]:
# ContextVar.set() is scoped to task, but changes to mutable data affect everyone
import asyncio
import contextvars
import copy

c = contextvars.ContextVar('c')
c.set({})

# this one works as expected
def contextupdate(k, v):
    print("got {}".format(c.get()))
    c.set({k: v})

# ...but this one breaks
def contextupdate2(k, v):
    d = c.get()
    print("got {}".format(d))
    d[k] = v
    # this is a red herring
    # we already modified the dict c is holding above
    c.set(d)

# ...but THIS one works
# `set` is local to the coroutine, but mutating the underlying dict affects everyone
def contextupdate3(k, v):
    d = copy.copy(c.get())
    print("got {}".format(d))
    d[k] = v
    c.set(d)
    
async def print_context(k, v, s):
    contextupdate3(k, v)
    await asyncio.sleep(s)
    print(c.get())
    print(list(contextvars.copy_context().items()))
    await asyncio.sleep(2)

    
async def main():
    await asyncio.gather(
        print_context('k1', 'v1', .2),
        print_context('k1', 'v2', .1)
    )
    
await main()

got {}
got {}
{'k1': 'v2'}
[(<ContextVar name='c' at 0x10915bbf0>, {'k1': 'v2'})]
{'k1': 'v1'}
[(<ContextVar name='c' at 0x10915bbf0>, {'k1': 'v1'})]


In [11]:
import asyncio
from opentelemetry import context
from opentelemetry.sdk.context.contextvars_context import ContextVarsContext

context.set_current(ContextVarsContext())
context.get_current().set_value('foo', 1)
context.get_current().get_value('foo')

from queue import Queue

q = Queue()

async def main():
    q.put(context.get_current().copy())
    q.put(context.get_current().copy())
    q.put(context.get_current().copy())
    
    f1 = worker(1)
    f2 = worker(2)
    
    asyncio.gather(f1, f2)

async def worker(v):
    print("worker {} start".format(v))
    await asyncio.sleep(.1)
    
    popped_context = q.get_nowait()
    print("popped context {}".format(popped_context))
    context.set_current(popped_context)
    await asyncio.sleep(.1)
    
    print("context in w{} is now {}".format(v, context.get_current()))
    print("setting {} in {}".format(v, context.get_current()))
    context.get_current().set_value('foo', v)
    await asyncio.sleep(.1 * v)


    
#     print("worker {} sleep".format(v))
#     # await asyncio.sleep(3 - v)
    
    print(v, context.get_current().get_value('foo'))
    print("worker {} done".format(v))


await main()

worker 1 start
worker 2 start
popped context <opentelemetry.sdk.context.contextvars_context.ContextVarsContext object at 0x10c8bcad0>
popped context <opentelemetry.sdk.context.contextvars_context.ContextVarsContext object at 0x10c8bca90>
context in w1 is now <opentelemetry.sdk.context.contextvars_context.ContextVarsContext object at 0x10c8bca90>
setting 1 in <opentelemetry.sdk.context.contextvars_context.ContextVarsContext object at 0x10c8bca90>
context in w2 is now <opentelemetry.sdk.context.contextvars_context.ContextVarsContext object at 0x10c8bca90>
setting 2 in <opentelemetry.sdk.context.contextvars_context.ContextVarsContext object at 0x10c8bca90>
1 1
worker 1 done
2 2
worker 2 done


In [45]:
import asyncio
from opentelemetry import context
from opentelemetry.sdk.context.contextvars_context import ContextVarsContext

from queue import Queue
from threading import Event

context.set_current(ContextVarsContext())

def tos(context):
    return "<Context at {}: {}>".format(
        hex(id(context)),
    ','.join(["{}={}".format(k, v.get(None)) for k, v in context._contextvars.items()])
    )

q = Queue()
run1 = Event()
run2 = Event()

async def main():
    q.put(context.get_current().copy())
    q.put(context.get_current().copy())
    
    asyncio.gather(a1(), a2())
    
async def a1():
    # (Step 1)
    context.set_current(q.get_nowait())
    context.get_current().set_value('k', 1)
    
    # Expected context is current, k=1
    print("[task 1] {}".format(tos(context.get_current())))
    
    # (GOTO 2)
    run2.set(); await asyncio.sleep(0); run1.wait();

    # (Step 3)
    # Current context has changed AND k=None because the call to `set` k=2 happened in the other task!
    print("[task 1] {}".format(tos(context.get_current())))


async def a2():   
    run2.wait()
    
    # (Step 2)  
    context.set_current(q.get_nowait())
    context.get_current().set_value('k', 2)
    
    # Expected context is current, k=2
    print("[task 2] {}".format(tos(context.get_current())))
    
    # (GOTO 3)
    run1.set(); await asyncio.sleep(0);

await main()

[task 1] <Context at 0x110349dd0: k=1>
[task 2] <Context at 0x110349250: k=2>
[task 1] <Context at 0x110349250: k=None>


In [45]:
import asyncio
from opentelemetry import context
from opentelemetry.sdk.context.contextvars_context import ContextVarsContext

from queue import Queue
from threading import Event

context.set_current(ContextVarsContext())

def tos(context):
    return "<Context at {}: {}>".format(
        hex(id(context)),
    ','.join(["{}={}".format(k, v.get(None)) for k, v in context._contextvars.items()])
    )

q = Queue()
run1 = Event()
run2 = Event()

async def main():
    q.put(context.get_current().copy())
    q.put(context.get_current().copy())
    
    asyncio.gather(work(1), work(2))
    
async def work(val):
    context.set_current(q.get_nowait())
    context.get_current().set_value('k', val)
    # At this point context.get_current() should be the one we popped from the stack, and current_context.get_value('k') should be val


async def a2():   
    run2.wait()
    
    # (Step 2)  
    context.set_current(q.get_nowait())
    context.get_current().set_value('k', 2)
    
    # Expected context is current, k=2
    print("[task 2] {}".format(tos(context.get_current())))
    
    # (GOTO 3)
    run1.set(); await asyncio.sleep(0);

await main()

[task 1] <Context at 0x110349dd0: k=1>
[task 2] <Context at 0x110349250: k=2>
[task 1] <Context at 0x110349250: k=None>


In [43]:
c = context.get_current()
c._contextvars

def print_context(context):
    print("<Context at {}: {}>".format(
        hex(id(context)),
    ','.join(["{}={}".format(k, v.get(None)) for k, v in context._contextvars.items()])
         ))
    
print_context(c)
cv = c._contextvars['k']

<Context at 0x1103cda90: k=None>


In [2]:
import asyncio
from queue import Queue

from opentelemetry import context
from opentelemetry.sdk.context.contextvars_context import ContextVarsContext

context.set_current(ContextVarsContext())

q = Queue()

async def main():
    q.put(context.get_current().copy())
    q.put(context.get_current().copy())
    asyncio.gather(work(1), work(2))

async def work(val):
    context.set_current(q.get_nowait())
    context.get_current().set_value('k', val)
    # At this point context.get_current() should be the one we popped from the
    # stack, and current_context.get_value('k') should be `val`

await main()

<Context at 0x106dc0790: k=1>
<Context at 0x107799710: k=2>
